Hướng dẫn chi tiết notebook API

- Mục tiêu: Thu thập dữ liệu các lần phóng của SpaceX qua API, chuẩn hoá và chuẩn bị cho phân tích.
- Luồng xử lý:
  1. Gọi API `/v4/launches/past` để lấy danh sách các lần phóng trong quá khứ.
  2. Chuyển JSON thành DataFrame bằng `pd.json_normalize`.
  3. Tạo `launch_df` với các cột quan trọng: `FlightNumber`, `Date`, `BoosterVersion`, `PayloadMass`, `Orbit`, `LaunchSite`, `Outcome`, `Flights`, `GridFins`, `Reused`, `Legs`, `LandingPad`.
  4. Viết các hàm tiện ích để truy xuất chi tiết từ các trường dạng danh sách (payloads, cores).
  5. Lọc chỉ giữ các lần phóng của Falcon 9 (nếu cần theo yêu cầu bài).
  6. Làm sạch dữ liệu: ép kiểu số, điền khuyết, chuẩn hoá giá trị boolean.

- Lưu ý:
  - Trường `payloads` và `cores` dạng danh sách đối tượng; cần duyệt phần tử đầu tiên hoặc hợp nhất theo yêu cầu.
  - `Outcome`: chuyển từ boolean `success` sang chuỗi `Success`/`Failure` để đồng nhất với các bài wrangling khác.
  - Sau khi làm sạch, có thể xuất CSV hoặc tiếp tục các bước phân tích/ trực quan hoá.

Chạy các cell theo thứ tự từ trên xuống để đảm bảo biến được khởi tạo đầy đủ.


# SpaceX API Data Collection Lab

In this lab, we will collect data from the SpaceX API and process it for analysis.

In [52]:
# Import các thư viện sử dụng trong notebook (API + xử lý dữ liệu)
import requests
import pandas as pd
import numpy as np

In [53]:
# Gọi API SpaceX để lấy danh sách các lần phóng trong quá khứ
# Lưu ý: API trả về ID cho rocket/payload/launchpad, ta sẽ map sang tên ở các cell sau
url = "https://api.spacexdata.com/v4/launches/past"
response = requests.get(url)
response.raise_for_status()
data = response.json()
print(f"Số bản ghi lấy được: {len(data)}")

Số bản ghi lấy được: 187


In [54]:
# Chuyển JSON thành DataFrame phẳng để dễ thao tác
# Cột như 'payloads', 'cores' vẫn là list ID/đối tượng; ta sẽ xử lý sau
df = pd.json_normalize(data)
print(f"Số cột: {df.shape[1]}")

Số cột: 43


In [55]:
# Xem nhanh các cột hiện có để định hướng xử lý
print("Các cột hiện có:")
print(df.columns.tolist())

Các cột hiện có:
['static_fire_date_utc', 'static_fire_date_unix', 'net', 'window', 'rocket', 'success', 'failures', 'details', 'crew', 'ships', 'capsules', 'payloads', 'launchpad', 'flight_number', 'name', 'date_utc', 'date_unix', 'date_local', 'date_precision', 'upcoming', 'cores', 'auto_update', 'tbd', 'launch_library_id', 'id', 'fairings.reused', 'fairings.recovery_attempt', 'fairings.recovered', 'fairings.ships', 'links.patch.small', 'links.patch.large', 'links.reddit.campaign', 'links.reddit.launch', 'links.reddit.media', 'links.reddit.recovery', 'links.flickr.small', 'links.flickr.original', 'links.presskit', 'links.webcast', 'links.youtube_id', 'links.article', 'links.wikipedia', 'fairings']


In [ ]:
# Xây dựng bảng dữ liệu chuẩn hoá theo các cột yêu cầu
# 1) Chuẩn bị mapping cho rocket (id -> name) và launchpad (id -> name)
rockets = {r['id']: r['name'] for r in requests.get("https://api.spacexdata.com/v4/rockets").json()}
launchpads = {p['id']: p['name'] for p in requests.get("https://api.spacexdata.com/v4/launchpads").json()}

# 2) Hàm hỗ trợ lấy payload mass/orbit từ ID (lấy phần tử đầu tiên nếu có nhiều payload)
def fetch_payload_attr(payload_ids, attr):
    try:
        if not payload_ids:
            return None
        pid = payload_ids[0]
        payload = requests.get(f"https://api.spacexdata.com/v4/payloads/{pid}").json()
        return payload.get(attr)
    except Exception:
        return None

# 3) Hàm lấy thuộc tính từ core đầu tiên
def get_core_attribute(core_list, attribute):
    try:
        if core_list and len(core_list) > 0:
            return core_list[0].get(attribute)
        return None
    except Exception:
        return None

# 4) Tạo DataFrame theo schema yêu cầu
launch_df = pd.DataFrame()
launch_df['FlightNumber'] = df['flight_number'].astype(int)
launch_df['Date'] = pd.to_datetime(df['date_utc']).dt.strftime('%Y-%m-%d')
launch_df['BoosterVersion'] = df['rocket'].map(rockets)  # map id -> tên (Falcon 9, Falcon Heavy, ...)
launch_df['PayloadMass'] = df['payloads'].apply(lambda lst: fetch_payload_attr(lst, 'mass_kg'))
launch_df['Orbit'] = df['payloads'].apply(lambda lst: fetch_payload_attr(lst, 'orbit'))
launch_df['LaunchSite'] = df['launchpad'].map(launchpads)
launch_df['Outcome'] = df['success'].apply(lambda x: 'Success' if x else 'Failure')
launch_df['Flights'] = df['cores'].apply(lambda x: get_core_attribute(x, 'flight'))
launch_df['GridFins'] = df['cores'].apply(lambda x: get_core_attribute(x, 'grid_fins'))
launch_df['Reused'] = df['cores'].apply(lambda x: get_core_attribute(x, 'reused'))
launch_df['Legs'] = df['cores'].apply(lambda x: get_core_attribute(x, 'legs'))
launch_df['LandingPad'] = df['cores'].apply(lambda x: get_core_attribute(x, 'landpad'))

# 5) Chỉ giữ Falcon 9 theo yêu cầu bài (slide)
launch_df = launch_df[launch_df['BoosterVersion'] == 'Falcon 9'].reset_index(drop=True)
# Đánh lại số thứ tự FlightNumber bắt đầu từ 1 sau khi lọc Falcon 9
launch_df['FlightNumber'] = range(1, len(launch_df) + 1)

print("\n5 dòng đầu của Falcon 9 sau khi chuẩn hoá:")
print(launch_df.head())

# Lưu tạm sang df để các cell sau dùng đồng nhất
df = launch_df.copy()

In [ ]:
# Clean and format the data
# Convert numeric columns to appropriate types
df['FlightNumber'] = df['FlightNumber'].astype(int)
df['PayloadMass'] = pd.to_numeric(df['PayloadMass'], errors='coerce')

# Calculate mean of PayloadMass
mean_payload_mass = df['PayloadMass'].mean()
# Replace NULL values in PayloadMass with the mean
df['PayloadMass'].fillna(mean_payload_mass, inplace=True)

# Convert boolean columns to proper format
df['GridFins'] = df['GridFins'].map({True: 'True', False: 'False'})
df['Reused'] = df['Reused'].map({True: 'True', False: 'False'})
df['Legs'] = df['Legs'].map({True: 'True', False: 'False'})

# Format the display
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

# Display the cleaned dataset
print("\nCleaned Falcon 9 launches data:")
print(df[['FlightNumber', 'Date', 'BoosterVersion', 'PayloadMass', 'Orbit', 
          'LaunchSite', 'Outcome', 'Flights', 'GridFins', 'Reused', 'Legs', 'LandingPad']])


Cleaned Falcon 9 launches data:
     FlightNumber        Date BoosterVersion   PayloadMass Orbit  \
0               6  2010-06-04       Falcon 9   8117.574038   LEO   
1               7  2010-12-08       Falcon 9   8117.574038   LEO   
2               8  2012-05-22       Falcon 9    525.000000   LEO   
3               9  2012-10-08       Falcon 9    400.000000   ISS   
4              10  2013-03-01       Falcon 9    677.000000   ISS   
..            ...         ...            ...           ...   ...   
174           183  2022-09-05       Falcon 9  13260.000000  VLEO   
175           184  2022-09-11       Falcon 9  13260.000000  VLEO   
176           185  2022-09-17       Falcon 9  13260.000000  VLEO   
177           186  2022-09-24       Falcon 9  13260.000000  VLEO   
178           187  2022-10-05       Falcon 9   8117.574038   ISS   

       LaunchSite  Outcome  Flights GridFins Reused   Legs  \
0    CCSFS SLC 40  Success        1      NaN  False  False   
1    CCSFS SLC 40  Success

C:\Users\letan\AppData\Local\Temp\ipykernel_29668\209583333.py:9: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['PayloadMass'].fillna(mean_payload_mass, inplace=True)


In [ ]:
# Functions to get additional data
def getBoosterVersion(rocket_id):
    url = f"https://api.spacexdata.com/v4/rockets/{rocket_id}"
    response = requests.get(url)
    return response.json()['name']

def getLaunchSite(launch_site_id):
    url = f"https://api.spacexdata.com/v4/launchpads/{launch_site_id}"
    response = requests.get(url)
    return response.json()['name']

def getPayloadData(payload_id):
    url = f"https://api.spacexdata.com/v4/payloads/{payload_id}"
    response = requests.get(url)
    return response.json()

def getCoreData(core_id):
    url = f"https://api.spacexdata.com/v4/cores/{core_id}"
    response = requests.get(url)
    return response.json()

In [ ]:
# Display the first few rows of the processed dataset
df.head()

,FlightNumber,Date,BoosterVersion,PayloadMass,Orbit,LaunchSite,Outcome,Flights,GridFins,Reused,Legs,LandingPad
0,6,2010-06-04,Falcon 9,8117.574038,LEO,CCSFS SLC 40,Success,1,NaN,False,False,None
1,7,2010-12-08,Falcon 9,8117.574038,LEO,CCSFS SLC 40,Success,1,NaN,False,False,None
2,8,2012-05-22,Falcon 9,525.000000,LEO,CCSFS SLC 40,Success,1,NaN,False,False,None
3,9,2012-10-08,Falcon 9,400.000000,ISS,CCSFS SLC 40,Success,1,NaN,False,False,None
4,10,2013-03-01,Falcon 9,677.000000,ISS,CCSFS SLC 40,Success,1,NaN,False,False,None
